<a href="https://colab.research.google.com/github/hemanthsaimanne/Oilpalm-image-classification-Python3-Tensorflow/blob/main/palmoil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from zipfile import ZipFile
file_name=os.path.join('/content/datasetzip.zip')
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 

    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

File Name                                             Modified             Size
dataset/not_ripen/                             2020-07-05 20:32:52            0
dataset/not_ripen/20200704_152422.jpg          2020-07-05 20:32:36      5810855
dataset/not_ripen/20200704_152515.jpg          2020-07-05 20:32:34      5266282
dataset/not_ripen/20200704_152519.jpg          2020-07-05 20:32:28      4243268
dataset/not_ripen/20200704_152522.jpg          2020-07-05 20:32:32      4280887
dataset/not_ripen/20200704_152543.jpg          2020-07-05 20:28:50      4932376
dataset/not_ripen/20200704_152548.jpg          2020-07-05 20:28:28      4569974
dataset/not_ripen/20200704_152556.jpg          2020-07-05 20:28:52      3209730
dataset/not_ripen/20200704_152612.jpg          2020-07-05 20:28:28      3034942
dataset/not_ripen/20200704_152618.jpg          2020-07-05 20:28:30      3009290
dataset/not_ripen/20200704_152619.jpg          2020-07-05 20:28:16      3002066
dataset/not_ripen/20200704_152620.jpg   

In [ ]:
BATCH_SIZE = 10
IMG_SHAPE  = 224

In [ ]:
dataset_path = '/content/dataset'

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,validation_split=0.2)

train_data_gen = image_gen.flow_from_directory(dataset_path,
                                               batch_size=BATCH_SIZE,
                                               target_size=(IMG_SHAPE,IMG_SHAPE),
                                               class_mode='binary',
                                               subset='training')
validation_data_gen = image_gen.flow_from_directory(dataset_path,
                                               batch_size=BATCH_SIZE,
                                               target_size=(IMG_SHAPE,IMG_SHAPE),
                                               class_mode='binary',
                                               subset='validation')


Found 14 images belonging to 2 classes.
Found 3 images belonging to 2 classes.


In [ ]:
module_selection = ("mobilenet_v2_100_224", 224) #@param ["(\"mobilenet_v2_100_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (224,224)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

#BATCH_SIZE =  256#@param {type:"integer"}

Using https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4 with input size (224, 224)


In [ ]:
do_fine_tuning = False #@param {type:"boolean"}

In [ ]:
import tensorflow_hub as hub

print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
   hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
   tf.keras.layers.Dense(9999, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
                          #kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

Building model with https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4
Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_9 (KerasLayer)   multiple                  2257984   
_________________________________________________________________
dense_26 (Dense)             multiple                  12808719  
_________________________________________________________________
dense_27 (Dense)             multiple                  20000     
Total params: 15,086,703
Trainable params: 12,828,719
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_9 (KerasLayer)   multiple                  2257984   
_________________________________________________________________
dense_26 (Dense)             multiple                  12808719  
_________________________________________________________________
dense_27 (Dense)             multiple                  20000     
Total params: 15,086,703
Trainable params: 12,828,719
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
model.fit(train_data_gen,
    validation_data = validation_data_gen,
    epochs = 3)

Epoch 1/3
2/2 [==============================] - 2s 934ms/step - loss: 7.6560 - accuracy: 0.6429 - val_loss: 7.6539 - val_accuracy: 0.6667
Epoch 2/3
2/2 [==============================] - 4s 2s/step - loss: 7.6560 - accuracy: 0.6429 - val_loss: 7.6539 - val_accuracy: 0.6667
Epoch 3/3
2/2 [==============================] - 2s 1s/step - loss: 7.6560 - accuracy: 0.6429 - val_loss: 7.6539 - val_accuracy: 0.6667


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(np.argmax(classes[0]))
  print(fn)
  print(classes)

Saving 20200704_152422.jpg to 20200704_152422 (2).jpg


OSError: ignored

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
from google.colab import files
files.download("/content/saved_model")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/saved_model/my_model")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>